In [ ]:
import os
import json
from PIL import Image

def define_areas(width, height):
    x_left_split = 0.30 * width
    x_right_split = 0.70 * width
    y_mid_split = 0.50 * height
    
    rect_front  = (0, 0, width, y_mid_split)
    rect_left   = (0, y_mid_split, x_left_split, height)
    rect_ground = (x_left_split, y_mid_split, x_right_split, height)
    rect_right  = (x_right_split, y_mid_split, width, height)
    
    return {
        "front":  rect_front,
        "left":   rect_left,
        "ground": rect_ground,
        "right":  rect_right
    }

def point_in_rect(px, py, rect):
    xmin, ymin, xmax, ymax = rect
    return (xmin <= px <= xmax) and (ymin <= py <= ymax)

def get_area_label(cx, cy, areas_dict):
    for label, rect in areas_dict.items():
        if point_in_rect(cx, cy, rect):
            return label
    return None

def main():
    JSON_FOLDER = "E:/Situational Awareness/Last Dataset/test/Position H type improve/json"
    FRAMES_FOLDER = "E:/Situational Awareness/Last Dataset/test/Position H type improve/frames"

    for json_file in os.listdir(JSON_FOLDER):
        if not json_file.endswith(".json"):
            continue
        
        video_name = os.path.splitext(json_file)[0]  # e.g. "video1"
        frames_path = os.path.join(FRAMES_FOLDER, video_name)

        json_path = os.path.join(JSON_FOLDER, json_file)
        
        # Load the JSON
        with open(json_path, "r") as f:
            data = json.load(f)
        
        for frame_dict in data:
            image_id = frame_dict.get("image_id", "")
            image_path = os.path.join(frames_path, image_id)
            
            if not os.path.exists(image_path):
                # Skip if missing image
                continue
            
            # Load the image to get dimensions
            with Image.open(image_path) as img:
                width, height = img.size
            
            # Create the 4 area rectangles for THIS image size
            areas = define_areas(width, height)
            
            # Now label each object
            graph = frame_dict.get("graph", {})
            nodes = graph.get("nodes", [])
            
            for node in nodes:
                if len(node) < 2:
                    continue
                obj_dict = node[1]
                
                # Make sure we have a position list
                if "position" not in obj_dict:
                    obj_dict["position"] = []
                
                # 1) If bounding box exists
                if "boxes" in obj_dict and len(obj_dict["boxes"]) == 4:
                    x1, y1, x2, y2 = obj_dict["boxes"]
                    cx = (x1 + x2) / 2.0
                    cy = (y1 + y2) / 2.0
                    area_label = get_area_label(cx, cy, areas)
                    if area_label and area_label not in obj_dict["position"]:
                        obj_dict["position"].append(area_label)
                
                # 2) If there's a 'point' instead
                if "point" in obj_dict and len(obj_dict["point"]) == 2:
                    px, py = obj_dict["point"]
                    area_label = get_area_label(px, py, areas)
                    if area_label and area_label not in obj_dict["position"]:
                        obj_dict["position"].append(area_label)
        
        # Save the updated JSON
        with open(json_path, "w") as f:
            json.dump(data, f, indent=2)

if __name__ == "__main__":
    main()
